# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("output_data/cities.csv")

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key = g_key)

In [4]:
locations = cities_df[["Lat", "Lng"]]

In [5]:
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout)

heat_layer = gmaps.heatmap_layer(locations, weights = cities_df["Humidity"],
                                dissipating = False, max_intensity= 100,
                                point_radius = 2.5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Using the example suggestions for ideal weather conditions

# A max temperature lower than 80 degrees but higher than 70

ideal_cities_df = cities_df[(cities_df["Max Temp"] > 70) & (cities_df["Max Temp"] < 80)]

# Wind speed less than 10 mph

ideal_cities_df = ideal_cities_df[(ideal_cities_df["Wind Speed"] < 5)]

# Zero cloudiness less than 10%

ideal_cities_df = ideal_cities_df[(ideal_cities_df["Cloudiness"] == 0)]
ideal_cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
49,Hamilton,39.1834,-84.5333,77.00,63,0,1.01,US,1660436582
88,Fairbanks,64.8378,-147.7164,75.09,58,0,3.44,US,1660436713
307,Westport,41.1415,-73.3579,74.75,58,0,3.44,US,1660436965
377,Mastic Beach,40.7668,-72.8521,73.98,59,0,3.44,US,1660437142
426,Palaia Fokaia,37.7200,23.9463,79.75,64,0,4.90,GR,1660437214
441,Shihezi,44.3000,86.0333,78.58,30,0,1.79,CN,1660437236
536,Olhão,37.0286,-7.8411,72.28,59,0,3.44,PT,1660437376


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_dict = {
    "Hotel Name" : [],
    "City" : [],
    "Country" : [],
    "Lat" : [],
    "Lng" : []
}

In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

target_search = "Hotel"

target_radius = 5000

target_type = "hotel"

In [9]:
i = 0
while i < len(ideal_cities_df):
    
    target_coordinates = f"{ideal_cities_df.iloc[i]['Lat']}, {ideal_cities_df.iloc[i]['Lng']}"

    parameters = {
        "location" : target_coordinates,
        "keyword": target_search,
        "radius" : target_radius,
        "type" : target_type,
        "key" : g_key
    }
    
    try:
        
        response = requests.get(base_url, params = parameters)
        
        hotel_data = response.json()
        
        hotel_dict["Hotel Name"].append(hotel_data["results"][0]["name"])
        hotel_dict["Lat"].append(hotel_data["results"][0]["geometry"]["location"]["lat"])
        hotel_dict["Lng"].append(hotel_data["results"][0]["geometry"]["location"]["lng"])
        hotel_dict["City"].append(ideal_cities_df.iloc[i]["City"])
        hotel_dict["Country"].append(ideal_cities_df.iloc[i]["Country"])
        
    except:
        
        print(f"No hotels found within {target_radius}m of {ideal_cities_df.iloc[i]['City']}...")
    
    i += 1

In [10]:
hotel_df = pd.DataFrame(hotel_dict)
hotel_df

,Hotel Name,City,Country,Lat,Lng
0,Graduate Cincinnati,Hamilton,US,39.136690,-84.507522
1,SpringHill Suites by Marriott Fairbanks,Fairbanks,US,64.843613,-147.720575
2,Residence Inn by Marriott Norwalk,Westport,US,41.097896,-73.419226
3,Smith Point Motel,Mastic Beach,US,40.751449,-72.871015
4,Plaza Resort Hotel,Palaia Fokaia,GR,37.723835,23.944509
5,Shihezi Binle Hotel,Shihezi,CN,44.321120,86.045876
6,Pure Formosa Concept Hotel,Olhão,PT,37.027356,-7.839671


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
                            info_box_content = hotel_info)

fig.add_layer(markers)

# Display figure

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…